In [ ]:
from music21 import *
import pandas as pd

# Load the MIDI file
midi_file = converter.parse('data/midi_dataset/ABBA/Super_Trouper.mid')

In [ ]:

# Create a dictionary to store the notes, chords, and tempo for each part
part_data = {}

# Initialize the part number to 1
part_number = 1

# Iterate over each part in the MIDI file
for i, part in enumerate(midi_file.parts):
    
    # Get the name of the instrument that plays this part, or use a default name if None
    instrument_name = part.getInstrument().instrumentName if part.getInstrument() else f'part_{part_number}'
    
    # If the instrument name is already in the dictionary, modify it to make it unique
    while instrument_name in part_data:
        part_number += 1
        instrument_name = f'part_{part_number}'
    
    # Create a list to store the notes, chords, and tempo for this part
    part_notes = []
    part_chords = []
    part_tempo = None
    tempo_list = []
    
    # Iterate over each element (note, chord, or metadata) in this part
    for element in part.flat:
        
        # Check if the element is a note
        if isinstance(element, note.Note):
            part_notes.append(element)
            
        # Check if the element is a chord
        elif isinstance(element, chord.Chord):
            part_chords.append(element)
            
        # Check if the element is a tempo indication
        elif isinstance(element, tempo.MetronomeMark):
            tempo_list.append(element.number)
    
    # Calculate the average tempo for this part if there are any tempo indications
    if tempo_list:
        part_tempo = sum(tempo_list) / len(tempo_list)
    
    # Add the notes, chords, and tempo for this part to the dictionary
    part_data[instrument_name] = {'notes': part_notes, 'chords': part_chords, 'tempo': part_tempo}
    
    # Increment the part number
    part_number += 1


In [ ]:
df=pd.DataFrame(part_data)


In [ ]:
for i, part in enumerate(midi_file.parts):
    print()

In [ ]:
df

#### Getting Something in the format of part_data to paly

In [ ]:
from music21 import *

# Create a new MIDI file
midi_stream = stream.Score()

# Iterate over each instrument in the part data
for instrument_name, instrument_data in part_data.items():

    # Create a new part for this instrument
    part = stream.Part()

    # Set the instrument for this part
    if instrument_name != 'None':
        part.insert(instrument.Instrument(instrument_name))

    # Add the notes to this part
    for note_obj in instrument_data['notes']:
        part.append(note_obj)

    # Add the chords to this part
    for chord_obj in instrument_data['chords']:
        part.append(chord_obj)

    # Add the tempo indication to this part
    if instrument_data['tempo'] is not None:
        part.append(tempo.MetronomeMark(number=instrument_data['tempo']))

    # Add this part to the MIDI file
    midi_stream.append(part)

# Write the MIDI file to disk
midi_stream.write('midi', fp='output9.mid')


NEW Approach

In [2]:
from music21 import *

# Load the MIDI file
midi_file = converter.parse("data/midi_dataset/ABBA/Super_Trouper.mid")

# Get the total length of the MIDI file in seconds
total_length = midi_file.duration.quarterLength * (60 / midi_file.getTempo().bpm)

# Create a dictionary to store the notes, chords, and tempo for each part
part_data = {}

# Iterate over each part in the MIDI file
for i, part in enumerate(midi_file.parts):
    
    # Get the name of the instrument that plays this part, or use a default name if None
    instrument_name = part.getInstrument().instrumentName if part.getInstrument() else f'part_{i}'
    
    # If the instrument name is already in the dictionary, modify it to make it unique
    while instrument_name in part_data:
        i += 1
        instrument_name = f'part_{i}'
    
    # Create a list to store the notes and chords for this part
    part_notes = []
    part_chords = []
    
    # Iterate over each element (note, chord, or metadata) in this part
    for element in part.flat:
        
        # Check if the element is a note
        if isinstance(element, note.Note):
            part_notes.append(element)
            
        # Check if the element is a chord
        elif isinstance(element, chord.Chord):
            part_chords.append(element)
    
    # Add the notes and chords for this part to the dictionary
    part_data[instrument_name] = {'notes': part_notes, 'chords': part_chords}
    
    # If this part is the melody, add a 'melody' tag to its entry in the dictionary
    if i == 0:
        part_data[instrument_name]['tag'] = 'melody'
        
    # If this part is the bass, add a 'bass' tag to its entry in the dictionary
    elif part.getInstrument() and part.getInstrument().instrumentName in ['Electric Bass', 'Acoustic Bass', 'Bass Guitar']:
        part_data[instrument_name]['tag'] = 'bass'
        
    # If this part is a harmonic accompaniment, add a 'harmonics' tag to its entry in the dictionary
    elif part.getInstrument() and part.getInstrument().instrumentName in ['Acoustic Guitar', 'Electric Guitar', 'Harpsichord', 'Piano', 'Organ']:
        part_data[instrument_name]['tag'] = 'harmonics'
        
    # Otherwise, add a 'misc' tag to its entry in the dictionary
    else:
        part_data[instrument_name]['tag'] = 'misc'

# Manipulate the part_data dictionary as needed

# Create a new MIDI file from the manipulated data
new_midi_file = midi.translate.music21ObjectToMidiFile(midi_file)

# Save the new MIDI file
new_midi_file.open("Outputs/output10.mid", "wb")
new_midi_file.write()
new_midi_file.close()

# Play the new MIDI file
play = midi.player.Player()
play.setTempo(midi_file.getTempo().bpm)
play.open(filename="Outputs/output10.mid")
play.play()


AttributeError: 'Score' object has no attribute 'getTempo'

In [3]:
import music21

# Load the MIDI file
midi_file = music21.converter.parse('data/midi_dataset/ABBA/Super_Trouper.mid')

# Get the total length of the MIDI file in seconds
total_length = midi_file.duration.quarterLength * (60 / midi_file.parts[0].getTempo().bpm)

# Create a dictionary to store the notes, chords, and tempo for each part
part_data = {}

# Initialize the part number to 1
part_number = 1

# Iterate over each part in the MIDI file
for i, part in enumerate(midi_file.parts):
    
    # Get the name of the instrument that plays this part, or use a default name if None
    instrument_name = part.getInstrument().instrumentName if part.getInstrument() else f'part_{part_number}'
    
    # If the instrument name is already in the dictionary, modify it to make it unique
    while instrument_name in part_data:
        part_number += 1
        instrument_name = f'part_{part_number}'
    
    # Create a list to store the notes, chords, and tempo for this part
    part_notes = []
    part_chords = []
    part_tempo = None
    
    # Get the length of the part in seconds
    part_length = float(part.duration.quarterLength * (60 / part.getTempo().bpm))
    
    # Calculate the tempo for this part so that it fills the entire length of the MIDI file
    part_tempo = int(round(60 / (part_length / total_length)))
    
    # Iterate over each element (note, chord, or metadata) in this part
    for element in part.flat:
        
        # Check if the element is a note
        if isinstance(element, music21.note.Note):
            part_notes.append(element)
            
        # Check if the element is a chord
        elif isinstance(element, music21.chord.Chord):
            part_chords.append(element)
    
    # Add the notes, chords, and tempo for this part to the dictionary
    part_data[instrument_name] = {'notes': part_notes, 'chords': part_chords, 'tempo': part_tempo}
    
    # Increment the part number
    part_number += 1


AttributeError: 'Part' object has no attribute 'getTempo'

In [4]:
import music21

# Load the MIDI file
midi_file = music21.converter.parse('data/midi_dataset/ABBA/Super_Trouper.mid')

# Get the total length of the MIDI file in seconds
total_length = midi_file.duration.quarterLength * (60 / get_tempo(midi_file))

# Define a function to get the tempo from a MIDI file
def get_tempo(midi_file):
    for event in midi_file.parts[0].flat:
        if 'SetTempo' in event.classes:
            return event.tempo.bpm
    # If no tempo event was found, return a default value of 120 bpm
    return 120

# Create a dictionary to store the notes, chords, and tempo for each part
part_data = {}

# Initialize the part number to 1
part_number = 1

# Iterate over each part in the MIDI file
for i, part in enumerate(midi_file.parts):
    
    # Get the name of the instrument that plays this part, or use a default name if None
    instrument_name = part.getInstrument().instrumentName if part.getInstrument() else f'part_{part_number}'
    
    # If the instrument name is already in the dictionary, modify it to make it unique
    while instrument_name in part_data:
        part_number += 1
        instrument_name = f'part_{part_number}'
    
    # Create a list to store the notes, chords, and tempo for this part
    part_notes = []
    part_chords = []
    part_tempo = None
    
    # Get the length of the part in seconds
    part_length = float(part.duration.quarterLength * (60 / get_tempo(midi_file)))
    
    # Calculate the tempo for this part so that it fills the entire length of the MIDI file
    part_tempo = int(round(60 / (part_length / total_length)))
    
    # Iterate over each element (note, chord, or metadata) in this part
    for element in part.flat:
        
        # Check if the element is a note
        if isinstance(element, music21.note.Note):
            part_notes.append(element)
            
        # Check if the element is a chord
        elif isinstance(element, music21.chord.Chord):
            part_chords.append(element)
    
    # Add the notes, chords, and tempo for this part to the dictionary
    part_data[instrument_name] = {'notes': part_notes, 'chords': part_chords, 'tempo': part_tempo}
    
    # Increment the part number
    part_number += 1


NameError: name 'get_tempo' is not defined

In [5]:
from music21 import *

# Define a function to get the tempo from a MIDI file
def get_tempo(midi_file):
    for i in midi_file.flat:
        if 'MetronomeMark' in i.classes:
            return i.getQuarterBPM()
    return None

# Load the MIDI file
midi_file = converter.parse('data/midi_dataset/ABBA/Super_Trouper.mid')

# Get the total length of the MIDI file in seconds
total_length = midi_file.duration.quarterLength * (60 / get_tempo(midi_file))

# Create a dictionary to store the notes, chords, and tempo for each part
part_data = {}
for i, part in enumerate(midi_file.parts):
    part_data[f'part_{i+1}'] = {'notes': [], 'chords': [], 'tempo': get_tempo(midi_file)}

    # Iterate through the notes and chords in the part
    for element in part.flat:
        if 'Note' in element.classes:
            part_data[f'part_{i+1}']['notes'].append(element)
        elif 'Chord' in element.classes:
            part_data[f'part_{i+1}']['chords'].append(element)
